In [1]:
# magic reload
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')
from analysis.analysis_utils import *
from analysis.plot_utils import *
from datasets import load_dataset

### Accuracy of QA expert

In [2]:
qa_df = prepare_df(types=['qa'])
filter = {'config_model_name_qa': 'deepseek/deepseek-v3.1-terminus', 'config_dataset_name_qa': 'Idavidrein/gpqa', 'config_dataset_subset_qa': 'gpqa_diamond', 'config_dataset_split_qa': 'train'}
qa_df_deepseek_31_terminus = qa_df[qa_df.apply(lambda row: all(row[col] == val for col, val in filter.items()), axis=1)]
qa_df_deepseek_31_terminus.shape


(196, 30)

In [4]:
qa_df_deepseek_31_terminus.columns

Index(['run_id_qa', 'datetime_qa', 'question_idx_qa', 'question_qa',
       'options_qa', 'correct_idx_qa', 'raw_model_response_qa',
       'parsed_model_response_qa', 'prompt_qa', 'token_usage_qa',
       'record_id_qa', 'prompt_template_qa', 'internal_model_reasoning_qa',
       'internal_model_reasoning_details_qa', 'success_qa', 'error_message_qa',
       'config_dataset_name_qa', 'config_dataset_subset_qa',
       'config_dataset_split_qa', 'config_model_name_qa',
       'config_temperature_qa', 'config_num_questions_qa',
       'config_random_seed_qa', 'config_num_choices_qa',
       'config_max_threads_qa', 'config_specific_question_idxs_qa',
       'config_rerun_qa', 'options_str_qa', 'parsed_answer_qa',
       'is_correct_qa'],
      dtype='object')

In [11]:
print(qa_df_deepseek_31_terminus['is_correct_qa'].mean().round(2))

0.77


### Analyze debate

In [3]:
debate_df = prepare_df(types=['debates'])
filter = {'run_id_debates': 'z1qae3w'}
debate_df_4_choice = debate_df[debate_df.apply(lambda row: all(row[col] == val for col, val in filter.items()), axis=1)]
debate_df_4_choice.shape


(154, 33)

In [16]:
debate_df_4_choice['debate_history_debates'].apply(lambda x: sum_reasoning_tokens_over_turns(x)).value_counts()

debate_history_debates
0    154
Name: count, dtype: int64

In [36]:
debate_df_4_choice.columns

Index(['config_dataset_name_debates', 'config_dataset_split_debates',
       'config_dataset_subset_debates', 'config_debater_model_debates',
       'config_debater_reasoning_effort_debates',
       'config_debater_reasoning_max_tokens_debates',
       'config_debater_temperature_debates', 'config_judge_model_debates',
       'config_judge_temperature_debates', 'config_max_output_tokens_debates',
       'config_max_threads_debates', 'config_num_choices_debates',
       'config_num_questions_debates', 'config_num_turns_debates',
       'config_private_reasoning_word_limit_debates',
       'config_private_scratchpad_debates',
       'config_public_argument_word_limit_debates',
       'config_random_seed_debates', 'correct_idx_debates', 'datetime_debates',
       'debate_history_debates', 'debate_history_text_debates',
       'error_message_debates', 'judge_verdict_debates', 'options_debates',
       'options_str_debates', 'prompt_template_debates',
       'prompt_templates_debates', 'que

In [ ]:
tot_issues = 0
for idx, row in debate_df_4_choice.iterrows():
    debate_history = row['debate_history_debates']
    for turn in debate_history:
        public_argument = turn['parsed_response']['public_argument'].lower()
        if 'private' in public_argument:
            print('='*80)
            print(f"record_id: {row['record_id_debates']}, debater_idx: {turn['debater_idx']}, turn: {turn['turn']}, success: {row['success_debates']}")
            print('='*80)
            print(public_argument)
            tot_issues += 1
        if 'wrong' in public_argument:
            print('='*80)
            print(f"issue_type: wrong in argument, record_id: {row['record_id_debates']}, debater_idx: {turn['debater_idx']}, turn: {turn['turn']}, success: {row['success_debates']}")
            print('='*80)
            print(public_argument)
            tot_issues += 1
print(f"\n\ntotal issues: {tot_issues}")


record_id: f23k876, debater_idx: 0, turn: 0, success: 1.0
before any docking study, we must correctly define the ligand's structure. xantheraquin's multiple chiral centers and tautomeric forms mean it isn't one molecule, but a family of distinct 3d structures with different shapes and chemical properties. docking is highly sensitive to this. a wrong chiral form will not fit the target; a wrong tautomer will form incorrect hydrogen bonds.

therefore, the most crucial step is to analyze all possible forms, but intelligently prioritize. we use computational chemistry to generate the plausible tautomers and enantiomers. then, we apply filters based on physicochemical properties—like predicted pka, energy in aqueous solution, and known pharmacophore patterns—to identify which forms are most likely to be biologically active. this ensures our docking studies target the *relevant* molecular species.

opposing approaches are premature. studying adme is useless if the molecule doesn't bind. runn